# Coursera Capstone Project

**Introduction/Business Problem:** We want to be able to find the best locations to start a restaurant in the Bay Area (the region encompassing San Francisco, CA and the cities and towns around it). We want to be able to attract the most customers and based on previous and nearby restaurants, we want to see if we can be successful in that area.

**Data:** We are going to get the Foursquare data for the Bay Area. From the venue list, we can calculate clusters at which most venues are located. Based on that data, we are going to identify the most popular areas for restaurants and food stores to begin, as they have the highest chance of attracting customers in those areas. We are going to use the data of the venues to cluster them into groups to find the busiest and most business filled areas in the Bay Area to find the best location for us to start a new business. 